<a href="https://colab.research.google.com/github/giholv/TechWeather/blob/main/Convers%C3%A3o_de_tabelas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Carregar o arquivo CSV
df = pd.read_csv('/content/drive/MyDrive/consumo_subprefeituras.csv', delimiter=';')

# Remover as linhas onde o ano é NaN
df = df[df['ANO'].notna()]

# Remover a coluna 'ANUAL' se ela existir
if 'ANUAL' in df.columns:
    df = df.drop('ANUAL', axis=1)

# Transformar colunas de meses em linhas usando pd.melt
df_melted = pd.melt(df, id_vars=['SUBPREFEITURAS', 'ANO'],
                    var_name='Mês',
                    value_name='Valor')

# Remover possíveis espaços extras nos nomes dos meses
df_melted['Mês'] = df_melted['Mês'].str.strip()

# Mapeamento de meses para números
mapa_meses = {
    'JAN': '01', 'FEV': '02', 'MAR': '03', 'ABR': '04',
    'MAI': '05', 'JUN': '06', 'JUL': '07', 'AGO': '08',
    'SET': '09', 'OUT': '10', 'NOV': '11', 'DEZ': '12'
}

# Garantir que a coluna 'ANO' seja do tipo inteiro
df_melted['ANO'] = df_melted['ANO'].astype(int)

# Adicionar a coluna 'Data' juntando o ano com o número do mês
df_melted['Data'] = df_melted['ANO'].astype(str) + '-' + df_melted['Mês'].map(mapa_meses)

# Converter a coluna 'Data' para o formato datetime
df_melted['Data'] = pd.to_datetime(df_melted['Data'], format='%Y-%m', errors='coerce')

# Verificar se há valores inválidos (NaT) após a conversão
invalid_dates = df_melted[df_melted['Data'].isna()]

# Preencher valores NaT na coluna Data com o último dia do mês
df_melted['Data'] = df_melted.apply(
    lambda row: row['Data'] if pd.notna(row['Data']) else pd.to_datetime(f"{row['ANO']}-{mapa_meses.get(row['Mês'], '01')}", format='%Y-%m') + pd.offsets.MonthEnd(1),
    axis=1
)

# Verificar se ainda há valores inválidos após o preenchimento
invalid_dates_final = df_melted[df_melted['Data'].isna()]

# Exibir os valores inválidos, se existirem
print(invalid_dates_final)

# # Exibir o DataFrame transformado
# consumo = df_melted.to_csv('/content/drive/MyDrive/consumo_subprefeituras_final.csv', index=False)

df_melted


Empty DataFrame
Columns: [SUBPREFEITURAS, ANO, Mês, Valor, Data]
Index: []


,SUBPREFEITURAS,ANO,Mês,Valor,Data
0,Aricanduva,2019,JAN,"54.366,70",2019-01-01
1,Butantã,2019,JAN,"137.325,87",2019-01-01
2,Campo Belo,2019,JAN,"89.278,75",2019-01-01
3,Capela Do Socorro,2019,JAN,"322.886,80",2019-01-01
4,Casa Verde,2019,JAN,"66.226,32",2019-01-01
...,...,...,...,...,...
1915,Sapopemba,2023,DEZ,"36.916,80",2023-12-01
1916,Sé,2023,DEZ,"72.224,55",2023-12-01
1917,Vila Maria/Vila Guilherme,2023,DEZ,"72.855,44",2023-12-01
1918,Vila Mariana,2023,DEZ,"73.088,67",2023-12-01


In [ ]:
import pandas as pd

# Carregar o arquivo CSV
data = pd.read_csv('/content/drive/MyDrive/crescimento_populacional_subprefeitura_CSV.csv', delimiter=';')

# Criar DataFrame
df = pd.DataFrame(data)

# Remover a coluna 'ANUAL' se ela existir
if 'CAGR' in df.columns:
    df = df.drop('CAGR', axis=1)

# Utilizar pd.melt para transformar colunas de anos em linhas
df_melted = pd.melt(df, id_vars=['Subprefeitura'],
                    var_name='Ano',
                    value_name='População')

# Extrair o ano da coluna "Ano" para deixar apenas os números
df_melted['Ano'] = df_melted['Ano'].str.extract('(\d{4})')

# Ordenar por subprefeitura e ano para facilitar a leitura

crescimento = df_melted.sort_values(by=['Subprefeitura', 'Ano'])
# Converter a coluna de consumo para numérico
crescimento['População'] = pd.to_numeric(crescimento['População'] , errors='coerce')
# # Exibir resultado
crescimento = crescimento.to_csv('/content/drive/MyDrive/crescimento_final.csv', index=False)

crescimento

In [ ]:
import pandas as pd

# Exemplo de dados de população
df_populacao = pd.read_csv('/content/drive/MyDrive/crescimento_final.csv')

# Exemplo de dados de consumo de energia
df_energia = pd.read_csv('/content/drive/MyDrive/consumo_subprefeituras_final.csv')

# Verificar colunas disponíveis
print("Colunas disponíveis em df_energia:", df_energia.columns)

# Renomear colunas
df_energia = df_energia.rename(columns={
    'SUBPREFEITURAS': 'Subprefeitura',
    'ANO': 'Ano',
    'Valor': 'Consumo de Energia (MWh)'
})

# Exibir o DataFrame de energia após renomeação
print(df_energia.head())

# Mesclar os DataFrames pela subprefeitura e ano
df_merged = pd.merge(df_populacao, df_energia, on=['Subprefeitura', 'Ano'])

# Verificar se a mesclagem foi bem-sucedida
if df_merged.empty:
    print("A mesclagem não retornou dados. Verifique as chaves de mesclagem.")
else:
    print("DataFrame mesclado:")
    print(df_merged.head())

    # Limpar a coluna de consumo de energia
    df_merged['Consumo de Energia (MWh)'] = (
        df_merged['Consumo de Energia (MWh)']
        .str.strip()  # Remove espaços em branco
        .str.replace('.', '', regex=False)  # Remove separadores de milhar
        .str.replace(',', '.', regex=False)  # Substitui vírgula por ponto
    )

    # Converter a coluna de consumo para numérico
    df_merged['Consumo de Energia (MWh)'] = pd.to_numeric(df_merged['Consumo de Energia (MWh)'], errors='coerce')

    # Exibir o DataFrame após a limpeza
    print("DataFrame após a limpeza:")
    print(df_merged.head())

    # Salvar o DataFrame resultante como um arquivo CSV
    df_merged.to_csv('/content/drive/MyDrive/df_consumoPopulacional.csv', index=False)


Colunas disponíveis em df_energia: Index(['SUBPREFEITURAS', 'ANO', 'Mês', 'Valor', 'Data'], dtype='object')
       Subprefeitura   Ano  Mês Consumo de Energia (MWh)        Data
0         Aricanduva  2019  JAN               54.366,70   2019-01-01
1            Butantã  2019  JAN              137.325,87   2019-01-01
2         Campo Belo  2019  JAN               89.278,75   2019-01-01
3  Capela Do Socorro  2019  JAN              322.886,80   2019-01-01
4         Casa Verde  2019  JAN               66.226,32   2019-01-01
DataFrame mesclado:
  Subprefeitura   Ano  População  Mês Consumo de Energia (MWh)        Data
0    Aricanduva  2019      266.5  JAN               54.366,70   2019-01-01
1    Aricanduva  2019      266.5  FEV               54.637,02   2019-02-01
2    Aricanduva  2019      266.5  MAR               53.807,28   2019-03-01
3    Aricanduva  2019      266.5  ABR               52.398,59   2019-04-01
4    Aricanduva  2019      266.5  MAI               53.405,27   2019-05-01
DataFram

In [ ]:
df_merged.head()

,Subprefeitura,Ano,População,Mês,Consumo de Energia (MWh),Data
0,Aricanduva,2019,266.5,JAN,54366.70,2019-01-01
1,Aricanduva,2019,266.5,FEV,54637.02,2019-02-01
2,Aricanduva,2019,266.5,MAR,53807.28,2019-03-01
3,Aricanduva,2019,266.5,ABR,52398.59,2019-04-01
4,Aricanduva,2019,266.5,MAI,53405.27,2019-05-01


In [ ]:
# Analisar a correlação entre o crescimento populacional e o consumo de energia

# Calculate the correlation between 'População' from 'crescimento' and 'Valor' from 'consumo'
correlation = df_merged['População'].corr( df_merged['Consumo de Energia (MWh)'])

# Exibir correlação
print(correlation)

-0.21848750319557822
